In [3]:
import sys
sys.path.insert(0,'../models/')
sys.path.insert(0,'../datasets/')
sys.path.insert(0,'..')

import pandas as pd
import numpy as np
import json
from subprocess import Popen, PIPE, STDOUT
import re
from collections import defaultdict

import tensorflow as tf
import tqdm
from models import PropbankEncoder
import config

INPUT_DIR = '../datasets/binaries/'
PROPBANK_GLO50_PATH = '{:}deep_glo50.pickle'.format(INPUT_DIR)


<h1><center>Structured Y Network CWIS SRL (BR)</center></h1>

<center>In this notebook we solve the semantic role labeling task using structured predictions networks.</center>

## 1. Builds a "human friendly" version of the dataset

In [4]:
dfgs = pd.read_csv('../datasets/csvs/gs.csv', index_col=0, sep=',', encoding='utf-8')
column_files = [
    '../datasets/csvs/column_chunks/chunks.csv',
    '../datasets/csvs/column_predmarker/predicate_marker.csv',
    '../datasets/csvs/column_shifts_ctx_p/form.csv',
    '../datasets/csvs/column_shifts_ctx_p/gpos.csv',
    '../datasets/csvs/column_shifts_ctx_p/lemma.csv',
    '../datasets/csvs/column_t/t.csv',
    '../datasets/csvs/column_iob/iob.csv'
]

for col_f in column_files:
    _df = pd.read_csv(col_f, index_col=0, encoding='utf-8')
    dfgs = pd.concat((dfgs, _df), axis=1)

DISPLAY_COLUMNS = ['ID', 'P', 'FORM', 'ARG', 'T', 
                   'CHUNK_ID', 'CHUNK_START', 'CHUNK_FINISH', 'CHUNK_LEN', 'CHUNK_CANDIDATE_ID']            
dfgs[DISPLAY_COLUMNS].head(33)    

,ID,P,FORM,ARG,T,CHUNK_ID,CHUNK_START,CHUNK_FINISH,CHUNK_LEN,CHUNK_CANDIDATE_ID
INDEX,,,,,,,,,,
0,1,1,Brasília,*,*,1,0,1,1,0
1,2,1,Pesquisa_Datafolha,(A0*,A0,2,1,4,3,35
2,3,1,publicada,*,A0,2,1,4,3,35
3,4,1,hoje,*),A0,2,1,4,3,35
4,5,1,revela,(V*),V,3,4,5,1,126
5,6,1,um,(A1*,A1,4,5,32,27,181
6,7,1,dado,*,A1,4,5,32,27,181
7,8,1,supreendente,*,A1,4,5,32,27,181
8,9,1,:,*,A1,4,5,32,27,181


## 2. Gets encodings

Propbank Encoder holds an indexed version of propbank dataset an answers to FOUR different dataformats: 
* CAT: this is the raw categorical data.
* EMB: tokens are embedding using GloVe embeddings.
* HOT: onehot encoding of the words and tokens.
* IDX: dense indexed representations.

In [5]:
# LOAD ENCODER
propbank_encoder = PropbankEncoder.recover(PROPBANK_GLO50_PATH)
db = propbank_encoder.db
lex2idx = propbank_encoder.lex2idx
idx2lex = propbank_encoder.idx2lex

# FOR TEXTUAL DATA ONLY
lex2tok = propbank_encoder.lex2tok
tok2idx = propbank_encoder.tok2idx
embeddings = propbank_encoder.embeddings

n_targets = len(lex2idx['T'])

In [6]:
print('attributes\t',
       len(db),
      '\n',             
      'records\t',
       len(db['ARG'].keys()))

attributes	 44 
 records	 141730


In [7]:
def filter_type(ds_type, db):
    '''Filters only records from train dataset
    '''
    ds_types = ('train', 'test', 'valid')
    if ds_type not in ds_types:
        _msg = 'ds_type must be in {:} got {:}'
        _msg = _msg.format(ds_types, ds_type)
        raise ValueError(_msg)
    elif ds_type in ('train',):
        lb = 0 
        ub = config.DATASET_TRAIN_SIZE
    elif ds_type in ('test',):        
        lb = config.DATASET_TRAIN_SIZE
        ub = lb + config.DATASET_VALID_SIZE         
    elif ds_type in ('valid',):                
        lb = config.DATASET_TRAIN_SIZE + config.DATASET_VALID_SIZE
        ub = lb + config.DATASET_TEST_SIZE         

    sel_keys_ = {key_ for key_, prop_ in db['P'].items() if prop_ > lb and prop_ <= ub}

    return {
                attr_:{ idx_: i_
                        for idx_, i_ in dict_.items() if idx_ in sel_keys_
                      }        
                for attr_, dict_  in db.items()
            }

def make_propositions_dict(db):
    '''Reindex db by propositions creating a nested dict in which the
        outer key is the proposition        
    '''
    
    triple_list = []
    prev_prop = -1
    for idx, prop in db['P'].items():
        if prev_prop != prop:
            if idx > 0:
                ub = idx-1
                triple_list.append((lb, ub, prev_prop))
            lb = idx
        prev_prop = prop
    triple_list.append((lb, ub, prev_prop))
            

        
    prop_set = set(db['P'].values())
    return { prop_:
                    {
                        attr_:{ idx_: dict_[idx_]
                                for idx_ in range(lb_, ub_ + 1, 1)
                          }        
                        for attr_, dict_ in db.items() if attr_ not in ('P',)
                    }
             for lb_, ub_, prop_ in  triple_list
            }, {prop_: ub_ - lb_ + 1 for lb_, ub_, prop_ in  triple_list}   


def numpfy_propositions_dict(prop_dict, proplen_dict):
    '''Converts inner dict examples into numpy arrays
    '''
    prop_dict_ = defaultdict(dict)    
    for prop, columns_dict in prop_dict.items():
        len_ = proplen_dict[prop]
        shape_ = (len_, 1)
        for column, values_dict in columns_dict.items():
            tuple_list = [idx_value 
                          for idx_value in values_dict.items()]
            
            tuple_list = sorted(tuple_list, key=lambda x: x[0])            
            # Converts lexicon (raw/indexed) into token (embedded/indexed)
            if (('FORM' in column) or ('LEMMA' in column)):
                values_list = [tok2idx[lex2tok[idx2lex[column][tuple_[1]]]]                
                                   for tuple_ in tuple_list]
            else:
                values_list = [tuple_[1] for tuple_ in tuple_list]
            
            prop_dict_[prop][column]  = np.array(values_list).reshape(shape_)
    
    return prop_dict_        


In [8]:
traindb  = filter_type('train', db)
print('attributes\t',
       len(traindb),
      '\n',             
      'records\t',
       len(traindb['ARG'].keys()),
       '\n',             
      'vocab\t',
        max([form for _, form in traindb['FORM'].items()]))

attributes	 44 
 records	 123846 
 vocab	 13289


In [9]:
prop_dict, proplen_dict = make_propositions_dict(traindb)
print('attributes\t',
       len(prop_dict[1]) + 1,
      '\n',             
      'records\t',
       sum([len(d['ARG']) for p, d in prop_dict.items()]),
        '\n',             
      'vocab\t',
        max([form for _, prop in prop_dict.items() for _, form in prop['FORM'].items()]))

attributes	 44 
 records	 123837 
 vocab	 13289


In [10]:
prop_dict1 = numpfy_propositions_dict(prop_dict, proplen_dict)
print('attributes\t',
       len(prop_dict1[1]) + 1,
      '\n',             
      'records\t',
       sum([len_ for _, len_ in proplen_dict.items()]),
        '\n',             
      'vocab\t',
        max([max(form) for _, prop in prop_dict1.items() for form in prop['FORM']]))

attributes	 44 
 records	 123837 
 vocab	 12037


In [11]:
def get_inputs(db1, propid):
    '''Generate inputs
    '''
    propdb = db1[propid] # nested dict of columns and idx value
    proplen = len(propdb['ID'])
    if 'CHUNK_SPACE' not in propdb:
        propdb['CHUNK_SPACE'] = generate_chunk_space(proplen)

    word    = propdb['FORM']
    ctx_p_left  = propdb['FORM_CTX_P-1']
    ctx_p0  = propdb['FORM_CTX_P+0']
    ctx_p_right  = propdb['FORM_CTX_P+1']
    
    marker  = propdb['MARKER']
    pos     = propdb['GPOS']
    chunk_type  = propdb['T']
    chunk_start, chunk_finish = propdb['CHUNK_SPACE']
    
    return word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish
            
def generate_chunk_space(n):
    '''Generates all possible spaces for chunks
    '''
    start_list = []
    end_list = []
    for i in range(n):
        for j in range(i,n,1):
            start_list.append(i)
            end_list.append(j+1)
    shape_ = (len(start_list), 1)
    start_ = np.array(start_list).reshape(shape_)
    finish_ = np.array(end_list).reshape(shape_)
    return start_, finish_
            

# def get_outputs(db1, propid):
#     ''' Generate outputs
#     '''
#     propdb_ = db1[propid] # nested dict of columns and idx value
#     plen_ = len(propdb_['ID'])
#     if 'OUTPUTS' not in propdb_: 
#         propdb_['OUTPUTS'] = propdb_['T'].reshape((plen_,))

#     return propdb_['OUTPUTS']

def get_outputs(db1, propid, n_targets):
    ''' Generate outputs
    '''
    propdb = db1[propid] # nested dict of columns and idx value
    if 'OUTPUTS' not in propdb: 
        # FIX THIS!!!
        id_type = np.concatenate(
            ( propdb['CHUNK_CANDIDATE_ID'], propdb['T']), axis=1
        )

        id_type = np.unique( id_type, axis=0)
        propdb['OUTPUTS'] = id_type[:,0] * n_targets + id_type[:,1]

    return propdb['OUTPUTS']

In [12]:
%%timeit
propid = 1120
word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1, propid)
y = get_outputs(prop_dict1, propid, n_targets)
# worst proposition 1120 size 92!

1.58 µs ± 377 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


 ## MODEL

In [13]:
propid = 1
word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1,  propid)
y = get_outputs(prop_dict1, propid, n_targets)
# proplen = proplen_dict[propid]
# y = y.reshape((proplen,1))
print(y)
_start  = np.repeat(chunk_start, n_targets)
_finish = np.repeat(chunk_finish, n_targets)
print(_start)
print(_finish)
print(list(zip(_start[y].flatten(), _finish[y].flatten())))


[    0  1261 20160  4571  6518]
[ 0  0  0 ..., 32 32 32]
[ 1  1  1 ..., 33 33 33]
[(0, 1), (1, 4), (32, 33), (4, 5), (5, 32)]


In [14]:
import struct_perc.colored_weighted_interval_scheduling as cwis
import struct_perc.weighted_interval_scheduling as wis
import struct_perc.utils as spu

 ## Tensorflow Graph

In [15]:
# vocab_size = len(lex2idx['FORM']) + 1
# embed_size = 50

# n_pos = len(lex2idx['GPOS'])
# # n_type = len(lex2idx['T'])
# n_classes  = len(lex2idx['T'])

# tf.reset_default_graph()

# # word index and gpos 
# tf_words = tf.placeholder(tf.int32, shape=(None,1))
# tf_pos = tf.placeholder(tf.int32, shape=(None,1))
# # t_x_type = tf.placeholder(tf.int32, shape=(None,1))

# # índices de inicio de intervalo
# tf_s = tf.placeholder(tf.int32, shape=(None,1))
# # índices de fim de intervalo
# tf_f = tf.placeholder(tf.int32, shape=(None,1))

# # replicamos os indicies de inicio e fim para cada classe de chunk possivel
# tf_sc = tf.reshape(
#       tf.tile(tf_s,  [1, n_classes]), [-1,1])
# tf_fc = tf.reshape(
#       tf.tile(tf_f,  [1, n_classes]), [-1,1])

# # n_features = (embed_size + n_pos + n_type)
# n_features = (embed_size + n_pos)
# # hidden_features = 300
# W_shape = (n_features, n_classes)
# EMBS = tf.constant(embeddings)
# # tf_token = tf.Variable(initial_value=None, expected_shape=(embed_size,), dtype=tf.float32, trainable=False)

# # geramos os paramteros do modelo
# with tf.variable_scope("model"):
#     W = tf.Variable(
#         tf.random_normal(W_shape, 0, 1/np.sqrt(n_features * n_classes), name='W')
#     )
#     b = tf.Variable(
#         tf.random_normal((n_classes,), 0, 1/np.sqrt(n_classes), name='b')
#     )
    

# # tf_token = tf.nn.embedding_lookup(tf_embeddings, id) 
# # Recuperamos os embeddings de cada palavra
# tf_word_features = tf.gather_nd(EMBS, tf_words)

# tf_pos_flat = tf.reshape(tf_pos, [-1])
# tf_pos_features = tf.one_hot(tf_pos_flat, depth=n_pos)

# # t_x_type_flat = tf.reshape(t_x_type,[-1])
# # t_type_features = tf.one_hot(t_x_type_flat, depth=n_type)

# # X = tf.concat((t_word_features,t_pos_features,t_type_features),axis=1)
# tf_tok_features = tf.concat((tf_word_features,tf_pos_features),axis=1)

# # a partir das features do intervalo computamos o score
# tf_scores = tf.matmul(tf_tok_features, W) + b

# tf_pred = tf.argmax(tf_scores, axis=1)


In [50]:
# Parameters
vocab_size = len(tok2idx)
embed_size = 50

n_pos = len(lex2idx['GPOS'])
n_classes = len(lex2idx['T'])
n_features = embed_size * 4 + 1 + n_pos

n_hidden = 100
filter_size = 5
filter_shape = (filter_size, n_features, n_hidden)
W_shape = (n_features, n_hidden)
b_shape = (1, n_hidden)

W_interval_shape = (2 * n_hidden, n_classes)
# W_shape = (hidden_features, n_classes)
b_interval_shape = (1, n_classes)

# word index 
X_words = tf.placeholder(tf.int64, shape=(None,1), name='word')

# predicate context index (left, predicate, right)
X_ctx_p_left = tf.placeholder(tf.int64, shape=(None,1), name='ctx_p_left')
X_ctx_p = tf.placeholder(tf.int64, shape=(None,1), name='ctx_p0')
X_ctx_p_right = tf.placeholder(tf.int64, shape=(None,1), name='ctx_p_right')

# POS tagging feature
X_pos = tf.placeholder(tf.int64, shape=(None,1), name='gpos')
X_marker = tf.cast( tf.placeholder(tf.int64, shape=(None,1), name='marker'), tf.float32 )
EMBS = tf.Variable(embeddings, trainable=False)

# Embedded representation
with tf.variable_scope("features"):
    EMBS_words = tf.gather_nd(EMBS, X_words, name='word_features')

    EMBS_ctx_pleft = tf.gather_nd(EMBS, X_ctx_p_left, name='EMBS_ctx_pleft')
    EMBS_ctx_p0 = tf.gather_nd(EMBS, X_ctx_p, name='EMBS_ctx_p0')
    EMBS_ctx_pright = tf.gather_nd(EMBS, X_ctx_p_right, name='EMBS_ctx_pright')

    X_pos_flat = tf.reshape(X_pos, [-1], name='gpos_flat')
    X_pos_onehot = tf.one_hot(X_pos_flat, depth=n_pos, name='gpos_onehot')

    X = tf.concat((EMBS_words, EMBS_ctx_pleft, EMBS_ctx_p0,
                   EMBS_ctx_pright, X_pos_onehot, X_marker),
                  axis=1, name='X')
    
with tf.variable_scope('hidden'):
    W = tf.Variable(tf.truncated_normal(W_shape, stddev=1.0 / np.sqrt(n_features * n_hidden)), name='W' )
    b = tf.Variable(tf.zeros(b_shape, dtype=tf.float32), name='b')
   
    Z = tf.nn.tanh( tf.matmul(X, W) + b, name='hidden_layer' )

# Those are the interval parameters
with tf.variable_scope("interval"):
    W_interval = tf.Variable(tf.random_normal(W_interval_shape, mean=0.0, stddev=1.0 / np.sqrt(1.0 * n_hidden * n_classes)), name='W_interval')
    b_interval = tf.Variable(tf.zeros(b_interval_shape, dtype=tf.float32), name='b_interval')
    
    # begin of interval
    i_s = tf.placeholder(tf.int32, shape=(None,1))
    # end of interval
    i_f = tf.placeholder(tf.int32, shape=(None,1))

    # interval for each class
#     i_s_class = tf.reshape(
#           tf.tile(i_s,  b_shape, (-1,1)))
#     i_f_class = tf.reshape(
#           tf.tile(i_f,  b_shape, (-1,1)))


i_f_score = tf.gather_nd(Z, i_f-1)
i_s_score = tf.gather_nd(Z, i_s)

i_score = tf.concat((i_f_score, i_s_score), axis=1)
scores_interval = tf.matmul(i_score, W_interval) + b_interval

scores_flat = tf.reshape(scores_interval, (-1,1))
scores_mean = tf.reduce_mean(scores_flat)
scores_diff = scores_flat - scores_mean

scores_std = tf.sqrt(tf.reduce_sum(scores_diff * scores_diff))
scores_op = scores_diff /( scores_std + 1e-8 )


# a partir das features do intervalo computamos o score
# score_op = tf.matmul(X, W_interval, name='xW') + b_interval

# predict_op = tf.argmax(scores_op, axis=1, name='Prediction')

 ## Tensorflow test session

In [55]:
propid  = 1
words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1, propid)
y = get_outputs(prop_dict1, propid, n_targets)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    out_scores_interval = sess.run(scores_interval, feed_dict={
        X_words:words,
        X_ctx_p_left: ctx_p_left,
        X_ctx_p: ctx_p0,
        X_ctx_p_right: ctx_p_right,        
        X_marker: marker,                
        X_pos:gpos,
        i_s: chunk_start,
        i_f: chunk_finish
    })
print(out_scores_interval.shape)


(561, 36)


In [61]:
def pred(sess, x_words, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos, x_chunk_start, x_chunk_finish):
    scores = sess.run(scores_op,feed_dict={
        X_words: x_words,
        X_ctx_p_left: x_ctx_p_left,
        X_ctx_p: x_ctx_p0,
        X_ctx_p_right: x_ctx_p_right,        
        X_marker: x_marker,                
        X_pos:x_pos,
        i_s: x_chunk_start,
        i_f: x_chunk_finish
    })
    starts = np.repeat(x_chunk_start, n_classes).reshape((-1,1))
    ends = np.repeat(x_chunk_finish, n_classes).reshape((-1,1))
    
    ck_len = len(x_chunk_start)
    colors = np.array(list(np.arange(n_classes))*ck_len)
    
    r_int = cwis.compute_schedule(starts.flatten(), ends.flatten(), scores, colors) 
    r_ext = list(zip(starts[r_int].flatten(),ends[r_int].flatten(), colors[r_int].flatten()))
    return r_int, r_ext

## Testing prediction

In [62]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
p, pe = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_start, chunk_finish)
print(p)
print(pe)

[20168 20120 20008 19844 19664 19448 19222 18908 18584 18235 17839 17396
 16928 16444 15884 15308 14707 14059 13364 12644 11914 11107 10268  9415
  8530  7594  6596  5588  4564  3464  2348  1216     8]
[(32, 33, 8), (31, 32, 32), (30, 31, 28), (29, 30, 8), (28, 29, 8), (27, 28, 8), (26, 27, 34), (25, 26, 8), (24, 25, 8), (23, 24, 19), (22, 23, 19), (21, 22, 8), (20, 21, 8), (19, 20, 28), (18, 19, 8), (17, 18, 8), (16, 17, 19), (15, 16, 19), (14, 15, 8), (13, 14, 8), (12, 13, 34), (11, 12, 19), (10, 11, 8), (9, 10, 19), (8, 9, 34), (7, 8, 34), (6, 7, 8), (5, 6, 8), (4, 5, 28), (3, 4, 8), (2, 3, 8), (1, 2, 28), (0, 1, 8)]


## Training code

In [ ]:
# t_y_int = tf.placeholder(tf.int32, shape=(None,))
# t_y_rshp = tf.reshape(t_y_int, (-1,1))
# t_margin_int = tf.placeholder(tf.float32, shape=())

# t_margin_values = tf.ones(tf.shape(t_y_rshp))*t_margin_int
# t_margin_scores = tf.scatter_nd(t_y_rshp, -t_margin_values, tf.shape(t_scores_flat))
# t_scores_flat_w_margin = t_scores_flat + t_margin_scores

In [ ]:
# indices of the correct intervals
T = tf.placeholder(tf.int32, shape=(None,), name='T')
L = tf.placeholder(tf.int32, shape=(), name='L')
I = tf.to_int32(tf.range(L), name='indices')

T_flat = tf.reshape(T, (-1,1), name='T_flat')
Margin_interval = tf.placeholder(tf.float32)
Margin_values = tf.ones(tf.shape(T_flat)) * Margin_interval
Margin_scores_flat = tf.scatter_nd(T_flat, -Margin_values, tf.shape(scores_op))
Scores_flat_with_margin = scores_op + Margin_scores_flat


In [ ]:
# # índices dos intervalos computados pelo Weighted Interval Scheduling
# t_p_int = tf.placeholder(tf.int32, shape=(None,))
# # índices dos intervalos corretos
# # t_y_int = tf.placeholder(tf.int32, shape=(None,))

# # formatamos os indices dos intervalos preditos e corretos
# t_p_rshp = tf.reshape(t_p_int,(-1,1))
# # t_y_rshp = tf.reshape(t_y_int,(-1,1))

# # score dos intervalos preditos
# t_scores_int_p = tf.gather_nd(t_scores_flat_w_margin, t_p_rshp)
# # score dos intervalos corretos
# t_scores_int_y = tf.gather_nd(t_scores_flat_w_margin, t_y_rshp)

# ## função de custo do perceptron estruturado
# # WIS
# t_cost_int = tf.reduce_sum(t_scores_int_p) - tf.reduce_sum(t_scores_int_y)
# t_cost = t_cost_int

# # # gradiente descendente no custo do perceptron estruturado
# optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
# # optimizer = tf.train.GradientDescentOptimizer(0.003)
# train = optimizer.minimize(t_cost)

In [ ]:
# indices containing the predicted labels from Weighted Interval Scheduling
Y = tf.placeholder(tf.int32, shape=(None,), name='predictions')
Y_flat = tf.reshape(Y, (-1, 1))

# score da estrutura predita
ScoreY = tf.gather_nd(Scores_flat_with_margin, Y_flat, name='predicted_score')
# score da estrutura correta
ScoreT = tf.gather_nd(Scores_flat_with_margin, T_flat, name='target_score')

# função de custo do perceptron estruturado
# cost_op = tf.reduce_sum(ScoreY) - tf.reduce_sum(ScoreT)
# t_cost_int = tf.reduce_sum(t_scores_int_p) - tf.reduce_sum(t_scores_int_y)
# t_cost = t_cost_int

# gradiente descendente no custo do perceptron estruturado
optimizer = tf.train.GradientDescentOptimizer(0.01)
train_op = optimizer.minimize(cost_op)

In [110]:
predictions = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos)
print(y.shape)
print(predictions)

(33,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


 ## Can it memorize ?

In [117]:
for i in range(10000):
    
    predictions = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos)
    
    
    _, cost = sess.run([train_op, cost_op], feed_dict={
        X_words: words,
        X_ctx_p_left: ctx_p_left,
        X_ctx_p: ctx_p0,
        X_ctx_p_right: ctx_p_right,        
        X_marker: marker,        
        X_pos: gpos,
        Y: predictions,
        L: proplen_dict[propid],
        T:y})


    if i % 100 == 0:
        err = np.sum(y.flatten() != predictions.flatten())
        acc = 1 - err/len(predictions.flatten())
        print(acc, ' ', cost)
        if acc == 1:
            break

0.0606060606061   0.0
1.0   0.0


 ## The model reproduces the sentence

In [118]:
p = sess.run(predict_op, 
             feed_dict={
                X_words: words,
                X_ctx_p_left: ctx_p_left,
                X_ctx_p: ctx_p0,
                X_ctx_p_right: ctx_p_right,        
                X_marker: marker,        
                X_pos: gpos
             })
print(p)
print()
print(y)

[ 0  1  1  1 35  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  0]

[ 0  1  1  1 35  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  0]


## Training

In [123]:
sess= tf.Session()
sess.run(tf.global_variables_initializer())

epochs = 30
for i in range(epochs):
    total_err = 0
    total_size = 0
    nprops = len(proplen_dict)
    for propid in tqdm.tqdm(range(1, nprops + 1)):
        words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1,  propid)
        y = get_outputs(prop_dict1, propid)

        predictions = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos)

        _, cost = sess.run([train_op, cost_op], feed_dict={
            X_words:words,
            X_ctx_p_left: ctx_p_left,
            X_ctx_p: ctx_p0,
            X_ctx_p_right: ctx_p_right,        
            X_marker: marker,
            X_pos: gpos,
            Y:predictions,
            L: proplen_dict[propid],
            T:y})
        total_err += np.sum(predictions.flatten()!=y.flatten())
        total_size += len(predictions.flatten())

    print('epoch ', i, ' acc: ', 1 - total_err/total_size)

  1%|▏         | 73/5099 [00:00<00:06, 723.52it/s]

epoch  0  acc:  0.424138181642


  1%|▏         | 76/5099 [00:00<00:06, 751.57it/s]

epoch  1  acc:  0.447644888038


  1%|          | 62/5099 [00:00<00:08, 613.77it/s]

...

epoch  27  acc:  0.467978067944


  2%|▏         | 77/5099 [00:00<00:06, 763.13it/s]

epoch  28  acc:  0.465838158224


100%|██████████| 5099/5099 [00:07<00:00, 721.11it/s]

epoch  29  acc:  0.469019759846
